In [3]:
from icrl import *
from model import *
import pickle

d:\Anaconda3\envs\textgen\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin d:\Anaconda3\envs\textgen\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [4]:
# load trajectory pkl
traj_0 = pickle.load(open('data/linucb_trajectories_part0.pkl', 'rb'))
traj_1 = pickle.load(open('data/linucb_trajectories_part1.pkl', 'rb'))
traj_2 = pickle.load(open('data/linucb_trajectories_part2.pkl', 'rb'))
traj_3 = pickle.load(open('data/linucb_trajectories_part3.pkl', 'rb'))
traj_4 = pickle.load(open('data/linucb_trajectories_part4.pkl', 'rb'))

# integrate all trajectories
traj = []
traj.extend(traj_0)
traj.extend(traj_1)
traj.extend(traj_2)
traj.extend(traj_3)
traj.extend(traj_4)

In [21]:
import wandb

wandb.login(key = 'da9f2bbc59300d7434138182a83a9b35c0cea793') # login to wandb
wandb.init(project="icrl_pretrain", reinit=True)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


test_loss,█▄▃▂▂▁
train_loss,█▂▁▁▁▁
test_loss,2009.93385
train_loss,2068.32754


In [22]:
# split trajectory into train and test
import tqdm
from sklearn.model_selection import train_test_split

train_traj, test_traj = train_test_split(traj, test_size=0.2)

batch_size = 16
# batch_size = 32

# create dataset
train_dataset = TrajectoryDataset(train_traj, 10)
test_dataset = TrajectoryDataset(test_traj, 10)

# create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# set up model
# model = TransformerModel(embed_dim=70, num_heads=5)
config = {
        'horizon': 200,
        'dim':5,
        'act_num':10,
        'state_dim': 50,
        'action_dim': 10,
        'n_layer': 8,
        'n_embd': 32,
        'n_head': 4,
        'shuffle': True,
        'dropout': 0.2,
        'test': False,
        'act_type': 'relu',

    }
action_dim = config['action_dim']
model = Transformer(config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device {device}")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss(reduction='sum')
# Trainer
def trainer(model,train_dataloader, test_dataloader, optimizer, num_epochs=10):
    # DataLoader can handle batching and shuffling
    model.train()  # Set the model to training mode

    for epoch in range(num_epochs):
        print(f"Epoch {epoch}")
        epoch_loss = 0.0
        for batch in tqdm.tqdm(train_dataloader):
            # Unpack the data
            # tokens, action_labels = batch
            # print(batch['context_actions'].size())
            pred_actions = model(batch) # dimension: (batch_size, seq_len, action_dim)
            # print(pred_actions.shape)
            true_actions = batch['true_actions'] # dimension: (batch_size, seq_len)
            pred_actions_flat =  pred_actions.view(-1, action_dim)
            true_actions_flat = true_actions.view(-1)
            loss = loss_fn(pred_actions_flat, true_actions_flat)
            # loss.backward()
            # print(action_labels)
            # Reset the gradients in the optimizer
            optimizer.zero_grad()
            # # type
            # # Forward pass
            # logits = model(tokens)  # Your forward method should handle the causal mask internally
            
            # # Compute loss
            # loss = loss(logits, action_labels)
    
            # # Backward pass
            loss.backward()
            
            # Update parameters
            optimizer.step()
            
            epoch_loss += loss.item()
        epoch_loss /= len(train_dataloader)
        # Compute the test loss
        test_loss = 0.0
        model.eval()
        with torch.no_grad():
            for batch in tqdm.tqdm(test_dataloader):
                pred_actions = model(batch)
                true_actions = batch['true_actions']
                pred_actions_flat =  pred_actions.view(-1, action_dim)
                true_actions_flat = true_actions.view(-1)
                loss = loss_fn(pred_actions_flat, true_actions_flat)
                
                test_loss += loss.item()
        model.train()
        test_loss /= len(test_dataloader)
        # wandb logging
        wandb.log({"train_loss": epoch_loss, "test_loss": test_loss})
        print(f"Epoch {epoch} | Train Loss {epoch_loss} | Test Loss {test_loss}")


Using device cuda


In [23]:
trainer(model, train_dataloader, test_dataloader, optimizer, num_epochs=15)

Epoch 0


100%|██████████| 1250/1250 [00:34<00:00, 35.84it/s]


Epoch 0 | Train Loss 2924.667772167969 | Test Loss 1246.77803984375
Epoch 1


100%|██████████| 1250/1250 [00:35<00:00, 35.48it/s]


Epoch 1 | Train Loss 1181.0856947021484 | Test Loss 1125.112143408203
Epoch 2


...

Epoch 12 | Train Loss 859.4235528686523 | Test Loss 855.276259765625
Epoch 13


100%|██████████| 1250/1250 [00:31<00:00, 39.55it/s]


Epoch 13 | Train Loss 851.4043709594727 | Test Loss 845.0645205566407
Epoch 14


100%|██████████| 1250/1250 [00:31<00:00, 39.18it/s]

Epoch 14 | Train Loss 844.6609286987305 | Test Loss 836.3830283203125


In [15]:
# save model
torch.save(model.state_dict(), 'model/transformer_model_0626.pth')

# RandomChoice + Qtransformer

In [1]:
import wandb

wandb.login(key = 'da9f2bbc59300d7434138182a83a9b35c0cea793') # login to wandb
wandb.init(project="icrl_pretrain", reinit=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xjxyys. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Lenovo/.netrc


In [2]:
import pickle
# load trajectory pkl
traj_0 = pickle.load(open('data/RandomChoose_trajectories_part0.pkl', 'rb'))
traj_1 = pickle.load(open('data/RandomChoose_trajectories_part1.pkl', 'rb'))
traj_2 = pickle.load(open('data/RandomChoose_trajectories_part2.pkl', 'rb'))
traj_3 = pickle.load(open('data/RandomChoose_trajectories_part3.pkl', 'rb'))
traj_4 = pickle.load(open('data/RandomChoose_trajectories_part4.pkl', 'rb'))
# integrate all trajectories
traj = []
traj.extend(traj_0)
traj.extend(traj_1)
traj.extend(traj_2)
traj.extend(traj_3)
traj.extend(traj_4)


In [12]:
# split trajectory into train and test
import tqdm
from sklearn.model_selection import train_test_split
from icrl import *

train_traj, test_traj = train_test_split(traj, test_size=0.2)

# batch_size = 128
batch_size = 32

# create dataset
train_dataset = TrajectoryDataset(train_traj, 10)
test_dataset = TrajectoryDataset(test_traj, 10)

# create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [33]:
# evaluate model
from model import *
config = {
        'horizon': 200,
        'dim':5,
        'act_num':10,
        'state_dim': 50,
        'action_dim': 10,
        'n_layer': 8,
        'n_embd': 32,
        'n_head': 4,
        'shuffle': True,
        'dropout': 0.2,
        'test': False,
        'act_type': 'relu',
        'pred_q': True
    }
# train
action_dim = config['action_dim']

model = Transformer(config)
# copy the model
# target_model = Transformer(config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device {device}")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=2.5e-4, weight_decay=1e-4)
# optimizer = torch.optim.Adam(model.parameters(), l=1e-5, weight_decay=1e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=5e-3, weight_decay=1e-4)
# loss_fn = nn.CrossEntropyLoss(reduction='sum')
loss_fn = F.mse_loss
gamma = 0.99

Using device cuda


In [34]:
Q_trainer(model, train_dataloader, test_dataloader, optimizer, num_epochs=5, double=True, gamma=gamma)

100%|██████████| 625/625 [00:22<00:00, 27.18it/s]


Train Loss 4.942751865005493 | Test Loss 4.947106595611572
Epoch 1


100%|██████████| 625/625 [00:46<00:00, 13.57it/s]


Epoch 1 | Train Loss 2.8417072288513183 | Test Loss 3.452147011947632
Epoch 2


100%|██████████| 625/625 [00:46<00:00, 13.43it/s]


...

Epoch 3 | Train Loss 2.6036930936813354 | Test Loss 2.69954414024353
Epoch 4


100%|██████████| 625/625 [00:48<00:00, 13.02it/s]


Epoch 4 | Train Loss 2.6587734623909 | Test Loss 2.6575336460113523
Epoch 5


100%|██████████| 625/625 [00:46<00:00, 13.35it/s]

Epoch 5 | Train Loss 2.71507711725235 | Test Loss 2.6220011482238768


In [35]:
# save model
torch.save(model.state_dict(), 'model/Qtransformer_model_0706_2.pth')

# randomChoose + Transformer

In [11]:
import pickle
# load trajectory pkl
traj_0 = pickle.load(open('data/RandomChoose_trajectories_part0.pkl', 'rb'))
traj_1 = pickle.load(open('data/RandomChoose_trajectories_part1.pkl', 'rb'))
traj_2 = pickle.load(open('data/RandomChoose_trajectories_part2.pkl', 'rb'))
traj_3 = pickle.load(open('data/RandomChoose_trajectories_part3.pkl', 'rb'))
traj_4 = pickle.load(open('data/RandomChoose_trajectories_part4.pkl', 'rb'))
# integrate all trajectories
traj = []
traj.extend(traj_0)
traj.extend(traj_1)
traj.extend(traj_2)
traj.extend(traj_3)
traj.extend(traj_4)

MemoryError: 

In [13]:
# split trajectory into train and test
import tqdm
from sklearn.model_selection import train_test_split

train_traj, test_traj = train_test_split(traj, test_size=0.2)

batch_size = 16
# batch_size = 32

# create dataset
train_dataset = TrajectoryDataset(train_traj, 10)
test_dataset = TrajectoryDataset(test_traj, 10)

# create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# set up model
# model = TransformerModel(embed_dim=70, num_heads=5)
config = {
        'horizon': 200,
        'dim':5,
        'act_num':10,
        'state_dim': 50,
        'action_dim': 10,
        'n_layer': 8,
        'n_embd': 32,
        'n_head': 4,
        'shuffle': True,
        'dropout': 0.2,
        'test': False,
        'act_type': 'relu',
        'pred_q': False
        

    }
action_dim = config['action_dim']
model = Transformer(config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device {device}")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-4)
loss_fn = nn.CrossEntropyLoss(reduction='sum')

Using device cuda


In [14]:
trainer(model, train_dataloader, test_dataloader, optimizer, num_epochs=5)

Epoch 0


100%|██████████| 1250/1250 [00:38<00:00, 32.77it/s]


Epoch 0 | Train Loss 7369.991474316406 | Test Loss 7369.60325234375
Epoch 1


100%|██████████| 1250/1250 [00:36<00:00, 34.51it/s]


Epoch 1 | Train Loss 7368.959986425782 | Test Loss 7369.124048828125
Epoch 2


...

Epoch 2 | Train Loss 7368.804815234375 | Test Loss 7368.98579140625
Epoch 3


100%|██████████| 1250/1250 [00:31<00:00, 39.28it/s]


Epoch 3 | Train Loss 7368.713823730469 | Test Loss 7368.627779296875
Epoch 4


100%|██████████| 1250/1250 [00:32<00:00, 38.67it/s]

Epoch 4 | Train Loss 7368.6082125 | Test Loss 7368.526080078125


In [15]:
# save model
torch.save(model.state_dict(), 'model/transformer_model_random.pth')

# LinUCB + Qtransformer

In [1]:
# load trajectory pkl
import pickle
traj_0 = pickle.load(open('data/linucb_trajectories_part0.pkl', 'rb'))
traj_1 = pickle.load(open('data/linucb_trajectories_part1.pkl', 'rb'))
traj_2 = pickle.load(open('data/linucb_trajectories_part2.pkl', 'rb'))
traj_3 = pickle.load(open('data/linucb_trajectories_part3.pkl', 'rb'))
traj_4 = pickle.load(open('data/linucb_trajectories_part4.pkl', 'rb'))

# integrate all trajectories
traj = []
traj.extend(traj_0)
traj.extend(traj_1)
traj.extend(traj_2)
traj.extend(traj_3)
traj.extend(traj_4)

In [2]:
# split trajectory into train and test
import tqdm
from sklearn.model_selection import train_test_split
from icrl import *

train_traj, test_traj = train_test_split(traj, test_size=0.2)

batch_size = 16
# batch_size = 32

# create dataset
train_dataset = TrajectoryDataset(train_traj, 10)
test_dataset = TrajectoryDataset(test_traj, 10)

# create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

d:\Anaconda3\envs\textgen\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


bin d:\Anaconda3\envs\textgen\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


In [3]:
import wandb
wandb.login(key = 'da9f2bbc59300d7434138182a83a9b35c0cea793') # login to wandb
wandb.init(project="icrl_pretrain", reinit=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: xjxyys. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Lenovo/.netrc


In [4]:
from model import *
config = {
        'horizon': 200,
        'dim':5,
        'act_num':10,
        'state_dim': 50,
        'action_dim': 10,
        'n_layer': 8,
        'n_embd': 32,
        'n_head': 4,
        'shuffle': True,
        'dropout': 0.2,
        'test': False,
        'act_type': 'relu',
        'pred_q': True
    }
# train
action_dim = config['action_dim']
model = Transformer(config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device {device}")
model.to(device)

# optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-4)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)

loss_fn = F.mse_loss
gamma = 1
# Trainer
def Q_trainer(model, train_dataloader, test_dataloader, optimizer, num_epochs=10, double=False):
    # record the loss before training
    train_loss = 0.0
    test_loss = 0.0
    model.eval()
    with torch.no_grad():
        for batch in tqdm.tqdm(train_dataloader):
            Qvalues = model(batch) # dimension: (batch_size, seq_len, A)
            # pred_Qvalues = pred_Qvalues.gather(2, batch['context_actions'].unsqueeze(-1).long()) # get the Q value of the context actions
            next_Qvalues = Qvalues.max(dim=-1, keepdim=True)[0]
            next_Qvalues = torch.cat([next_Qvalues[:, 1:,:], torch.zeros((next_Qvalues.shape[0], 1, 1), device=next_Qvalues.device)], dim=1)
            pred_Qvalues =  Qvalues * batch['context_actions']
            pred_Qvalues =  pred_Qvalues.sum(dim=-1, keepdim=True)
            TD_target = batch['context_rewards'] + gamma * next_Qvalues
            loss = loss_fn(pred_Qvalues, TD_target)
            train_loss += loss.item()
        train_loss /= len(train_dataloader)

        for batch in tqdm.tqdm(test_dataloader):
            # pred_Qvalues = model(batch).gather(2, batch['context_actions'].unsqueeze(-1).long())
            Qvalues = model(batch)
            next_Qvalues = Qvalues.max(dim=-1, keepdim=True)[0]

            next_Qvalues = torch.cat([next_Qvalues[:, 1:, :], torch.zeros((next_Qvalues.shape[0], 1, 1), device=next_Qvalues.device)], dim=1)
            pred_Qvalues =  Qvalues * batch['context_actions']
            pred_Qvalues =  pred_Qvalues.sum(dim=-1, keepdim=True)

        
            # shifted_pred_Qvalues = torch.cat([pred_Qvalues[:, 1:, :], torch.zeros((pred_Qvalues.shape[0], 1, 1), device=pred_Qvalues.device)], dim=1)
            TD_target = batch['context_rewards'] + gamma * next_Qvalues
            loss = loss_fn(pred_Qvalues, TD_target)
            test_loss += loss.item()
        test_loss /= len(test_dataloader)
        print(f"Train Loss {train_loss} | Test Loss {test_loss}")
        wandb.log({"train_loss": train_loss, "test_loss": test_loss})
    # DataLoader can handle batching and shuffling
    model.train()  # Set the model to training mode

    if double:
        target_model = Transformer(config)
        target_model.load_state_dict(model.state_dict())
        target_model.eval()
        target_model.to(device)

    for epoch in range(1, num_epochs+1):
        print(f"Epoch {epoch}")
        epoch_loss = 0.0
        counter = 0
        for batch in tqdm.tqdm(train_dataloader):
            Qvalues = model(batch) # dimension: (batch_size, seq_len, A)
            pred_Qvalues = Qvalues * batch['context_actions']
            pred_Qvalues =  pred_Qvalues.sum(dim=-1, keepdim=True)
            
            with torch.no_grad():
                next_actions = model(batch).max(dim=-1, keepdim=True)[1]

                if double:
                    # use target model to get the next Q values
                    # next_Qvalues = target_model(batch).gather(2, next_actions.unsqueeze(-1).long())
                    next_Qvalues = target_model(batch).gather(2, next_actions.long())
                else:
                    next_Qvalues = Qvalues.max(dim=-1, keepdim=True)[0]
                next_Qvalues = torch.cat([next_Qvalues[:, 1:, :], torch.zeros((next_Qvalues.shape[0], 1, 1), device=next_Qvalues.device)], dim=1)
            # Note: batch['context_rewards'] should be in the shape (batch_size, seq_len, 1) to match pred_Qvalues
            TD_target = batch['context_rewards'] + gamma * next_Qvalues
            # Calculate the loss
            loss = loss_fn(pred_Qvalues, TD_target, reduction='mean')
            # Reset the gradients in the optimizer
            optimizer.zero_grad()
            # # Backward pass
            loss.backward()
            # Update parameters
            optimizer.step()
            
            epoch_loss += loss.item()
            counter += 1
            # if double:
            #     for param, target_param in zip(model.parameters(), target_model.parameters()):
            #         target_param.data = target_param.data * (1 - 0.01) + param.data * 0.01
            if double:
                if counter % 100 == 0:
                    target_model.load_state_dict(model.state_dict())
            
        epoch_loss /= len(train_dataloader)
        # Compute the test loss
        test_loss = 0.0
        model.eval()
        with torch.no_grad():
            for batch in tqdm.tqdm(test_dataloader):
                # pred_Qvalues = model(batch).gather(2, batch['context_actions'].unsqueeze(-1).long())
                Qvalues = model(batch)
                pred_Qvalues = Qvalues * batch['context_actions']
                pred_Qvalues =  pred_Qvalues.sum(dim=-1, keepdim=True)
                next_actions = model(batch).max(dim=-1, keepdim=True)[1]
                if double:
    
                    next_Qvalues = target_model(batch).gather(2, next_actions.long())
                else:
                    next_Qvalues = Qvalues.max(dim=-1, keepdim=True)[0]
                    
                next_Qvalues = torch.cat([next_Qvalues[:, 1:, :], torch.zeros((next_Qvalues.shape[0], 1, 1), device=next_Qvalues.device)], dim=1)
                # shifted_pred_Qvalues = torch.cat([pred_Qvalues[:, 1:, :], torch.zeros((pred_Qvalues.shape[0], 1, 1), device=pred_Qvalues.device)], dim=1)
                
                TD_target = batch['context_rewards'] + gamma * next_Qvalues

                loss = loss_fn(pred_Qvalues, TD_target)
                test_loss += loss.item()
        model.train()
        test_loss /= len(test_dataloader)
        # wandb logging
        wandb.log({"train_loss": epoch_loss, "test_loss": test_loss})
        print(f"Epoch {epoch} | Train Loss {epoch_loss} | Test Loss {test_loss}")

Using device cuda


In [5]:
Q_trainer(model, train_dataloader, test_dataloader, optimizer, num_epochs=5, double=True)

  0%|          | 0/5000 [00:00<?, ?it/s]

100%|██████████| 1250/1250 [00:44<00:00, 28.11it/s]


Train Loss 6.694190567207336 | Test Loss 6.6788156131744385
Epoch 1


100%|██████████| 1250/1250 [01:01<00:00, 20.17it/s]


Epoch 1 | Train Loss 4.526185640001297 | Test Loss 4.9170932235717775
Epoch 2


...

Epoch 3 | Train Loss 9.400312283039092 | Test Loss 4.173528666305542
Epoch 4


100%|██████████| 1250/1250 [00:59<00:00, 20.88it/s]


Epoch 4 | Train Loss 3.4564990319252016 | Test Loss 3.602992325973511
Epoch 5


100%|██████████| 1250/1250 [00:57<00:00, 21.68it/s]

Epoch 5 | Train Loss 3.3751141761779784 | Test Loss 3.736875250053406


In [6]:
# save model
torch.save(model.state_dict(), 'model/Qtransformer_model_ucb_0707.pth')